In [45]:
import import_ipynb
import pandas as pd
import numpy as np
from IPython.display import HTML, display

In [46]:
from P01_Pre_Processing import matches
from P01_Pre_Processing import deliveries

In [47]:
def generate_ipl_scorecard(match_id, deliveries=deliveries, matches=matches):
    match_data = matches[matches['Id'] == match_id].iloc[0]
    match_deliveries = deliveries[deliveries['Match_Id'] == match_id]
    
    team1 = match_data['Team1']
    team2 = match_data['Team2']
    
    # Handle toss decision and batting order
    if match_data['Toss_Decision'] == 'Bat':
        first_batting = match_data['Toss_Winner']
        second_batting = team2 if team1 == match_data['Toss_Winner'] else team1
    else:
        second_batting = match_data['Toss_Winner']
        first_batting = team2 if team1 == match_data['Toss_Winner'] else team1
    
    def generate_innings_data(innings_num, batting_team, bowling_team):
        innings_data = match_deliveries[
            (match_deliveries['Inning'] == innings_num) & 
            (match_deliveries['Batting_Team'] == batting_team)
        ].copy()
        
        if innings_data.empty:
            return None
        
        # Get all batsmen who faced at least one ball
        all_batsmen = innings_data['Batter'].unique()
        
        # Get batsmen who were actually dismissed
        dismissed_batsmen_data = innings_data[
            (innings_data['Is_Wicket'] == 1) & 
            (innings_data['Player_Dismissed'].notna())
        ]
        dismissed_batsmen = dismissed_batsmen_data['Player_Dismissed'].unique()
        
        # Get the last ball to see who was at crease
        last_ball = innings_data.iloc[-1]
        batsmen_at_crease = {last_ball['Batter'], last_ball['Non_Striker']}
        
        # Determine batting order by first appearance (simpler approach)
        batsmen_order = []
        seen_batsmen = set()
        
        for _, ball in innings_data.iterrows():
            batter = ball['Batter']
            if batter not in seen_batsmen:
                batsmen_order.append(batter)
                seen_batsmen.add(batter)
        
        # Batsman statistics with proper order - FIXED DISMISSAL LOGIC
        batsman_data = []
        total_wickets = innings_data['Is_Wicket'].sum()
        
        for batsman in batsmen_order:
            batsman_balls = innings_data[innings_data['Batter'] == batsman]
            runs = batsman_balls['Batsman_Runs'].sum()
            balls = len(batsman_balls)
            strike_rate = round((runs / balls * 100), 2) if balls > 0 else 0
            
            # Check if batsman was dismissed
            was_dismissed = batsman in dismissed_batsmen
            
            # Check if batsman was at crease at the end
            was_at_crease_at_end = batsman in batsmen_at_crease
            
            # FIXED: Determine dismissal text - ONLY mark as "not out" if at crease AND not dismissed
            if was_dismissed:
                # Get dismissal details
                dismissal_data = innings_data[
                    (innings_data['Player_Dismissed'] == batsman) & 
                    (innings_data['Is_Wicket'] == 1)
                ].iloc[0]
                
                bowler = dismissal_data['Bowler']
                dismissal = dismissal_data['Dismissal_Kind']
                fielder = dismissal_data['Fielder'] if pd.notna(dismissal_data['Fielder']) else ""
                
                if dismissal == 'Caught':
                    dismissal_text = f"c {fielder} b {bowler}"
                elif dismissal == 'Bowled':
                    dismissal_text = f"b {bowler}"
                elif dismissal == 'Lbw':
                    dismissal_text = f"lbw b {bowler}"
                elif dismissal == 'Run Out':
                    dismissal_text = f"run out ({fielder})" if fielder else "run out"
                elif dismissal == 'Stumped':
                    dismissal_text = f"st {fielder} b {bowler}"
                elif dismissal == 'Caught And Bowled':
                    dismissal_text = f"c & b {bowler}"
                elif dismissal == 'Hit Wicket':
                    dismissal_text = f"hit wicket b {bowler}"
                elif dismissal == 'Obstructing The Field':
                    dismissal_text = f"obstructing the field"
                elif dismissal == 'Retired Hurt':
                    dismissal_text = f"retired hurt"
                elif dismissal == 'Retired Out':
                    dismissal_text = f"retired out"
                else:
                    dismissal_text = f"{dismissal} b {bowler}"
            elif was_at_crease_at_end:
                # ONLY mark as "not out" if they were actually at the crease when innings ended
                dismissal_text = "not out"
            else:
                # For batsmen who weren't dismissed and weren't at crease at the end
                # This means they completed their innings but were not at crease when innings ended
                # In proper cricket scoring, they should NOT be marked as "not out"
                dismissal_text = ""  # Leave blank - they are considered "out" by virtue of not being at crease
        
            batsman_data.append({
                'Batsman': batsman,
                'Dismissal': dismissal_text,
                'Runs': runs,
                'Balls': balls,
                'SR': strike_rate
            })
        
        batsman_stats = pd.DataFrame(batsman_data)
        
        # Bowler statistics - exclude penalty runs from bowler figures
        bowler_data = []
        bowlers_used = innings_data['Bowler'].unique()
        
        for bowler in bowlers_used:
            bowler_balls = innings_data[innings_data['Bowler'] == bowler]
            
            # Calculate balls bowled (exclude wides and no-balls from ball count)
            valid_balls = bowler_balls[~bowler_balls['Extras_Type'].isin(['Wides', 'Noballs'])]
            balls_bowled = len(valid_balls)
            
            if balls_bowled == 0:
                continue
                
            # Calculate runs conceded (exclude penalty runs)
            runs_conceded = bowler_balls['Batsman_Runs'].sum() + bowler_balls['Extra_Runs'].sum()
            penalty_runs = bowler_balls[bowler_balls['Extras_Type'] == 'Penalty']['Extra_Runs'].sum()
            runs_conceded -= penalty_runs
            
            # Calculate wickets
            wickets = bowler_balls['Is_Wicket'].sum()
            
            # Calculate overs properly (4.1 means 4 overs 1 ball, not 4.1 overs)
            overs = balls_bowled // 6
            balls_in_over = balls_bowled % 6
            overs_display = f"{overs}.{balls_in_over}" if balls_in_over > 0 else f"{overs}"
            
            # Calculate economy rate
            total_overs = overs + (balls_in_over / 6)
            economy = round(runs_conceded / total_overs, 2) if total_overs > 0 else 0
            
            bowler_data.append({
                'Bowler': bowler,
                'Overs': overs_display,
                'Runs': runs_conceded,
                'Wickets': wickets,
                'Economy': economy
            })
        
        bowler_stats = pd.DataFrame(bowler_data)
        
        # Sort bowlers by wickets (most first) then by overs (most first)
        if not bowler_stats.empty:
            bowler_stats = bowler_stats.sort_values(['Wickets', 'Overs'], ascending=[False, False])
        
        # Calculate match totals
        extras_breakdown = innings_data.groupby('Extras_Type')['Extra_Runs'].sum()
        extras = extras_breakdown.sum()
        
        # Detailed extras breakdown
        extras_detail = []
        for extra_type in ['Wides', 'Noballs', 'Byes', 'Legbyes', 'Penalty']:
            if extra_type in extras_breakdown and extras_breakdown[extra_type] > 0:
                extras_detail.append(f"{extras_breakdown[extra_type]} {extra_type.lower()}")
        
        total_runs = innings_data['Total_Runs'].sum()
        total_wickets = innings_data['Is_Wicket'].sum()
        
        # Calculate valid balls for total overs (exclude wides and no-balls)
        valid_balls_total = innings_data[~innings_data['Extras_Type'].isin(['Wides', 'Noballs'])]
        total_balls_bowled = len(valid_balls_total)
        total_overs = total_balls_bowled // 6
        total_balls = total_balls_bowled % 6
        
        # Format overs properly (4.1 means 4 overs 1 ball)
        total_overs_display = f"{total_overs}.{total_balls}" if total_balls > 0 else f"{total_overs}"
        
        return {
            'batting_team': batting_team,
            'bowling_team': bowling_team,
            'batsmen': batsman_stats,
            'bowlers': bowler_stats,
            'extras': extras,
            'extras_detail': ', '.join(extras_detail),
            'total_runs': total_runs,
            'total_wickets': total_wickets,
            'total_overs': total_overs_display,
            'total_balls_bowled': total_balls_bowled
        }
    
    # Generate innings data
    innings1 = generate_innings_data(1, first_batting, second_batting)
    innings2 = generate_innings_data(2, second_batting, first_batting)
    
    # Handle match result with all edge cases
    def get_match_result():
        result = match_data['Result']
        winner = match_data['Winner']
        
        if pd.isna(result) or result == 'No Result':
            return "MATCH ABANDONED - NO RESULT", None
        
        if result == 'Tie':
            if match_data['Super_Over'] == 'Y':
                return "MATCH TIED - DECIDED BY SUPER OVER", winner
            else:
                return "MATCH TIED", None
        
        # Handle D/L method
        method_note = " (D/L method)" if match_data['Method'] == 'D/L' else ""
        
        if result == 'Runs':
            margin = match_data['Result_Margin']
            return f"{winner.upper()} WIN BY {margin} RUNS{method_note}", winner
        elif result == 'Wickets':
            margin = match_data['Result_Margin']
            return f"{winner.upper()} WIN BY {margin} WICKETS{method_note}", winner
        
        return "MATCH COMPLETED", winner
    
    result_text, winner = get_match_result()
    
    # Handle match type display
    match_type = match_data['Match_Type']
    match_type_display = {
        'League': 'LEAGUE MATCH',
        'Semi Final': 'SEMI FINAL',
        'Final': 'FINAL',
        '3Rd Place Play-Off': '3RD PLACE PLAY-OFF',
        'Qualifier 1': 'QUALIFIER 1',
        'Elimination Final': 'ELIMINATION FINAL',
        'Qualifier 2': 'QUALIFIER 2',
        'Eliminator': 'ELIMINATOR'
    }.get(match_type, match_type.upper())
    
    # CSS Styling (same as before)
    css_style = """
    <style>
    .ipl-scorecard {
        font-family: 'Arial', sans-serif;
        max-width: 1000px;
        margin: 20px auto;
        background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
        border-radius: 15px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        overflow: hidden;
    }
    
    .final-match {
        background: linear-gradient(135deg, #8B0000 0%, #FFD700 100%) !important;
    }
    
    .playoff-match {
        background: linear-gradient(135deg, #4B0082 0%, #9370DB 100%) !important;
    }
    
    .match-header {
        background: linear-gradient(45deg, #004ba0, #1976d2);
        color: white;
        padding: 20px;
        text-align: center;
        border-bottom: 3px solid #ffd700;
    }
    
    .final-header {
        background: linear-gradient(45deg, #B22222, #DC143C) !important;
    }
    
    .playoff-header {
        background: linear-gradient(45deg, #4B0082, #8A2BE2) !important;
    }
    
    .match-title {
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 5px;
        text-transform: uppercase;
        letter-spacing: 1px;
    }
    
    .match-subtitle {
        font-size: 16px;
        opacity: 0.9;
        margin-bottom: 5px;
    }
    
    .match-type {
        font-size: 14px;
        background: rgba(255,255,255,0.2);
        padding: 5px 10px;
        border-radius: 15px;
        display: inline-block;
        margin-top: 5px;
    }
    
    .innings-container {
        display: flex;
        flex-wrap: wrap;
        gap: 20px;
        padding: 20px;
    }
    
    .innings-card {
        flex: 1;
        min-width: 450px;
        background: white;
        border-radius: 10px;
        box-shadow: 0 5px 15px rgba(0,0,0,0.1);
        overflow: hidden;
    }
    
    .innings-header {
        background: linear-gradient(45deg, #d32f2f, #f44336);
        color: white;
        padding: 15px;
        font-weight: bold;
        font-size: 18px;
        text-align: center;
    }
    
    .team-score {
        background: #f8f9fa;
        padding: 15px;
        text-align: center;
        border-bottom: 2px solid #e9ecef;
    }
    
    .score-large {
        font-size: 28px;
        font-weight: bold;
        color: #d32f2f;
    }
    
    .score-over {
        font-size: 16px;
        color: #666;
        margin-top: 5px;
    }
    
    .section-title {
        background: #e9ecef;
        padding: 10px 15px;
        font-weight: bold;
        color: #495057;
        border-bottom: 1px solid #dee2e6;
    }
    
    .batting-table, .bowling-table {
        width: 100%;
        border-collapse: collapse;
        table-layout: fixed;
    }
    
    .batting-table th, .bowling-table th {
        background: #495057;
        color: white;
        padding: 12px 8px;
        text-align: left;
        font-size: 14px;
    }
    
    .batting-table td, .bowling-table td {
        padding: 10px 8px;
        border-bottom: 1px solid #e9ecef;
        font-size: 14px;
        word-wrap: break-word;
    }
    
    .batting-table tr:hover, .bowling-table tr:hover {
        background: #f8f9fa;
    }
    
    .not-out {
        color: #28a745;
        font-weight: bold;
    }
    
    .result-section {
        background: linear-gradient(45deg, #388e3c, #4caf50);
        color: white;
        padding: 25px;
        text-align: center;
        border-top: 3px solid #ffd700;
    }
    
    .tie-result {
        background: linear-gradient(45deg, #FF8C00, #FFA500) !important;
    }
    
    .no-result {
        background: linear-gradient(45deg, #696969, #808080) !important;
    }
    
    .result-text {
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 10px;
    }
    
    .mom-text {
        font-size: 18px;
        opacity: 0.9;
        margin-top: 10px;
    }
    
    .extras-total {
        background: #e3f2fd;
        padding: 10px 15px;
        border-top: 1px solid #bbdefb;
        font-weight: bold;
    }
    
    .extras-detail {
        font-size: 12px;
        color: #666;
        margin-top: 5px;
    }
    
    .dls-note {
        background: #fff3cd;
        color: #856404;
        padding: 10px;
        margin: 10px 0;
        border-radius: 5px;
        border-left: 4px solid #ffc107;
        font-size: 14px;
    }
    
    /* Column widths for better formatting */
    .batting-table th:nth-child(1), .batting-table td:nth-child(1) { width: 25%; }
    .batting-table th:nth-child(2), .batting-table td:nth-child(2) { width: 35%; }
    .batting-table th:nth-child(3), .batting-table td:nth-child(3) { width: 10%; }
    .batting-table th:nth-child(4), .batting-table td:nth-child(4) { width: 10%; }
    .batting-table th:nth-child(5), .batting-table td:nth-child(5) { width: 20%; }
    
    .bowling-table th:nth-child(1), .bowling-table td:nth-child(1) { width: 35%; }
    .bowling-table th:nth-child(2), .bowling-table td:nth-child(2) { width: 15%; }
    .bowling-table th:nth-child(3), .bowling-table td:nth-child(3) { width: 15%; }
    .bowling-table th:nth-child(4), .bowling-table td:nth-child(4) { width: 15%; }
    .bowling-table th:nth-child(5), .bowling-table td:nth-child(5) { width: 20%; }
    
    @media (max-width: 768px) {
        .innings-card {
            min-width: 100%;
        }
        .batting-table, .bowling-table {
            font-size: 12px;
        }
    }
    </style>
    """
    
    # Determine special styling for match type
    card_class = "ipl-scorecard"
    header_class = "match-header"
    result_class = "result-section"
    
    if 'Final' in match_type:
        card_class += " final-match"
        header_class += " final-header"
    elif match_type in ['Semi Final', 'Qualifier 1', 'Qualifier 2', 'Eliminator', 'Elimination Final', '3Rd Place Play-Off']:
        card_class += " playoff-match"
        header_class += " playoff-header"
    
    if 'Tie' in result_text:
        result_class += " tie-result"
    elif 'No Result' in result_text or 'ABANDONED' in result_text:
        result_class += " no-result"
    
    # Generate HTML content
    html_content = f"""
    {css_style}
    <div class="{card_class}">
        <div class="{header_class}">
            <div class="match-title">TATA IPL {match_data['Season']}</div>
            <div class="match-subtitle">MATCH {match_id} • {match_data['Venue']}</div>
            <div class="match-subtitle">{match_data['City']} • {pd.to_datetime(match_data['Date']).strftime('%d %b %Y')}</div>
            <div class="match-type">{match_type_display}</div>
        </div>
    """
    
    # Add DLS note if applicable
    if match_data['Method'] == 'D/L':
        html_content += f"""
        <div class="dls-note">
            <strong>Duckworth-Lewis-Stern Method Applied:</strong> Target adjusted due to weather interruptions
        </div>
        """
    
    html_content += """
        <div class="innings-container">
    """
    
    # Function to render innings
    def render_innings(innings, innings_num):
        if not innings:
            return f"""
            <div class="innings-card">
                <div class="innings-header">INNINGS {innings_num} DATA NOT AVAILABLE</div>
            </div>
            """
        
        html = f"""
            <div class="innings-card">
                <div class="innings-header">{innings['batting_team'].upper()}</div>
                <div class="team-score">
                    <div class="score-large">{innings['total_runs']}-{innings['total_wickets']}</div>
                    <div class="score-over">OVERS {innings['total_overs']}</div>
                </div>
                
                <div class="section-title">BATTING</div>
                <table class="batting-table">
                    <thead>
                        <tr>
                            <th>Batsman</th>
                            <th>Dismissal</th>
                            <th>R</th>
                            <th>B</th>
                            <th>SR</th>
                        </tr>
                    </thead>
                    <tbody>
        """
        
        # Add batsmen rows
        for _, batsman in innings['batsmen'].iterrows():
            not_out_class = "not-out" if "not out" in batsman['Dismissal'].lower() else ""
            html += f"""
                        <tr>
                            <td><strong>{batsman['Batsman']}</strong></td>
                            <td class="{not_out_class}">{batsman['Dismissal']}</td>
                            <td><strong>{batsman['Runs']}</strong></td>
                            <td>{batsman['Balls']}</td>
                            <td>{batsman['SR']}</td>
                        </tr>
            """
        
        html += f"""
                    </tbody>
                </table>
                <div class="extras-total">
                    Extras: {innings['extras']}
                    <div class="extras-detail">{innings['extras_detail']}</div>
                </div>
                
                <div class="section-title">BOWLING</div>
                <table class="bowling-table">
                    <thead>
                        <tr>
                            <th>Bowler</th>
                            <th>O</th>
                            <th>R</th>
                            <th>W</th>
                            <th>Econ</th>
                        </tr>
                    </thead>
                    <tbody>
        """
        
        # Add bowlers rows
        if not innings['bowlers'].empty:
            for _, bowler in innings['bowlers'].iterrows():
                html += f"""
                        <tr>
                            <td><strong>{bowler['Bowler']}</strong></td>
                            <td>{bowler['Overs']}</td>
                            <td>{bowler['Runs']}</td>
                            <td><strong>{bowler['Wickets']}</strong></td>
                            <td>{bowler['Economy']}</td>
                        </tr>
                """
        else:
            html += """
                        <tr>
                            <td colspan="5" style="text-align: center; padding: 20px;">No bowling data available</td>
                        </tr>
            """
        
        html += """
                    </tbody>
                </table>
            </div>
        """
        return html
    
    # Render both innings
    html_content += render_innings(innings1, 1)
    html_content += render_innings(innings2, 2)
    
    html_content += f"""
        </div>
        
        <div class="{result_class}">
            <div class="result-text">{result_text}</div>
    """
    
    # Add toss information
    html_content += f"""
            <div class="match-subtitle">
                Toss: {match_data['Toss_Winner']} chose to {match_data['Toss_Decision'].lower()}
            </div>
    """
    
    # Add Man of the Match if available
    if pd.notna(match_data['Player_Of_Match']):
        html_content += f'<div class="mom-text">PLAYER OF THE MATCH: {match_data["Player_Of_Match"]}</div>'
    
    # Add super over note if applicable
    if match_data['Super_Over'] == 'Y':
        html_content += '<div class="mom-text">• Match decided by Super Over</div>'
    
    html_content += """
        </div>
    </div>
    """
    
    return HTML(html_content)

In [48]:
def generate_all_scorecards(deliveries, matches, limit=5):
    """
    Generate styled scorecards for multiple matches
    """
    unique_matches = matches['id'].unique()
    
    for i, match_id in enumerate(unique_matches[:limit]):
        display(generate_ipl_scorecard(match_id))
        if i < limit - 1:
            display(HTML("<hr style='margin: 40px 0; border: 1px solid #ddd;'>"))

# Example usage:
# Display single match scorecard
scorecard_html = generate_ipl_scorecard(335982)
display(scorecard_html)

# Display multiple match scorecards
# generate_all_styled_scorecards(deliveries, matches, limit=3)

Batsman,Dismissal,R,B,SR
SC Ganguly,,10,12,83.33
BB McCullum,not out,158,77,205.19
RT Ponting,,20,20,100.0
DJ Hussey,,12,12,100.0
Mohammad Hafeez,not out,5,3,166.67
Bowler,O,R,W,Econ
Z Khan,4,38,1,9.5
AA Noffke,4,41,1,10.25
JH Kallis,4,52,1,13.0
P Kumar,4,41,0,10.25


In [49]:
deliveries[(deliveries['Match_Id'] == 335982) & (deliveries['Is_Wicket'] == 1)]

,Match_Id,Inning,Batting_Team,Bowling_Team,Over,Ball,Batter,Bowler,Non_Striker,Batsman_Runs,Extra_Runs,Total_Runs,Extras_Type,Is_Wicket,Player_Dismissed,Dismissal_Kind,Fielder
33,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,5,2,SC Ganguly,Z Khan,BB McCullum,0,0,0,NaN,1,Sc Ganguly,Caught,Jh Kallis
74,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,12,1,RT Ponting,JH Kallis,BB McCullum,0,0,0,NaN,1,Rt Ponting,Caught,P Kumar
106,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,17,1,DJ Hussey,AA Noffke,BB McCullum,0,0,0,NaN,1,Dj Hussey,Caught,Cl White
131,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,R Dravid,I Sharma,W Jaffer,0,0,0,NaN,1,R Dravid,Bowled,NaN
138,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,V Kohli,AB Dinda,W Jaffer,0,0,0,NaN,1,V Kohli,Bowled,NaN
154,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,4,5,JH Kallis,AB Agarkar,W Jaffer,0,0,0,NaN,1,Jh Kallis,Caught,M Kartik
157,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,5,2,W Jaffer,AB Dinda,CL White,0,0,0,NaN,1,W Jaffer,Caught,Rt Ponting
174,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,7,5,MV Boucher,SC Ganguly,CL White,0,0,0,NaN,1,Mv Boucher,Caught,M Kartik
177,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,8,2,B Akhil,AB Agarkar,CL White,0,0,0,NaN,1,B Akhil,Caught,Rt Ponting
183,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,8,8,CL White,AB Agarkar,AA Noffke,0,0,0,NaN,1,Cl White,Caught,Wp Saha
